# Предсказание активности молекул по отношению к таргету

*При подготовке ноутбука использовались данные из соревнования [Global AI Challenge](https://codenrock.com/contests/global-ai#/)* 

Целью данной задачи является предсказание активности молекулы лиганда по отношению к таргету - Covid 19

![](https://cloudfront.jove.com/files/media/science-education/science-education-thumbs/11513.jpg)

## План анализа данных:

  1. Загрузить данные для обучения
  2. Обработать данные перед обучением модели
  3. Обучить модель на обучающей выборке
  4. Загрузить и предобработать данные для тестирования
  5. Провалидировать модель на тестовой выборке


# 0. Установка и импорт библиотек

In [ ]:
# Grab Jaime's excellent condacolab package: https://github.com/jaimergp/condacolab
# Note: you should probably read the README file at that repo.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:34
🔁 Restarting kernel...


In [ ]:
!mamba install -c conda-forge rdkit chembl_structure_pipeline

Streaming output truncated to the last 5000 lines.
libbrotlicommon          [] (00m:00s) Waiting...
pytz                     [] (00m:00s) Waiting...
sqlite                   [] (00m:00s) Waiting...
python                   [] (00m:06s) Validating...
libglib                  [] (00m:05s) Decompressing...
libopenblas              [] (00m:05s) Waiting...
packaging                [] (00m:05s) Waiting...
pcre                     [] (00m:05s) Waiting...
matplotlib-base          [] (00m:05s) Waiting...
pixman                   [] (00m:05s) Waiting...
numpy                    [] (00m:05s) Waiting...
pycairo                  [] (00m:05s) Waiting...
python-dateutil          [] (00m:05s) Waiting...
rdkit                    [] (00m:05s) Validating...
sqlalchemy               [] (00m:05s) Waiting...
xorg-libice              [] (00m:05s) Waiting...
unicodedata2             [] (00m:05s) Waiting...
xorg-libxdmcp            [] (00m:05s) Waiting...
xorg-libxrender          [] (00m:05s) Waiting...
xorg-l

In [ ]:
!pip install  dgl
!pip install dgllife

     |████████████████████████████████| 4.4 MB 3.0 MB/s 
     |████████████████████████████████| 1.9 MB 42.0 MB/s 
     |████████████████████████████████| 38.1 MB 341 kB/s 
     |████████████████████████████████| 138 kB 5.3 MB/s 
     |████████████████████████████████| 24.8 MB 100.1 MB/s 
     |████████████████████████████████| 1.6 MB 53.2 MB/s 
     |████████████████████████████████| 306 kB 44.6 MB/s 
     |████████████████████████████████| 198 kB 49.0 MB/s 
     |████████████████████████████████| 829 kB 60.4 MB/s 
  Created wheel for dgllife: filename=dgllife-0.2.9-py3-none-any.whl size=219059 sha256=e084a5fccc8bc988347253eb7d8ab09a203d342d4547b20e67968410ef3e8931
  Stored in directory: /root/.cache/pip/wheels/34/83/11/9772c74b559d9182c9083362e8ba8b0201c4963e41e03859fe
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=19aefc779bd6f12dec52f5b80471288bdb0a44988b47c6785753e89e3c64546f
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 1. Загрузка данных

In [ ]:
!wget https://www.dropbox.com/s/48c34raijlxc0nw/train.csv
!wget https://www.dropbox.com/s/297trreazro8ivr/test_labels.csv

--2022-03-15 11:27:24--  https://www.dropbox.com/s/48c34raijlxc0nw/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/48c34raijlxc0nw/train.csv [following]
--2022-03-15 11:27:24--  https://www.dropbox.com/s/raw/48c34raijlxc0nw/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc91422700ec8f860163f5a6f131.dl.dropboxusercontent.com/cd/0/inline/Bhjrw3LCdA5dgoVa9nQhBqgpOugard4kaNi0b3MPtEndS9udwx0dBV3MA4i3g1ATJyyzcRLudRw1yBgk51KniKp8XJc0QusBGjeqA7d3f2L9OTu5_6WXEhfPjnChW9brZSvpkT3UF7wPHKjBeZtnostHFhW8Mg-j0HawBf6TbDBErg/file# [following]
--2022-03-15 11:27:25--  https://uc91422700ec8f860163f5a6f131.dl.dropboxusercontent.com/cd/0/inline/Bhjrw3LCdA5dgoVa9nQhBqgpOugard4kaNi0b3MPtEndS9udwx0dBV3MA4i3g1ATJyyzcRLudRw1y

In [ ]:
DATA_PATH = "./"
TRAIN_FILE = "train.csv"
TEST_FILE = "test_labels.csv"

SMILES_COLUMN = "smiles"
TARGET_COLUMN = "Active"

In [ ]:
class Data():
    def __init__(self, data_dir: str = DATA_PATH, filename: str = TRAIN_FILE):
        self.data_dir = data_dir
        self.filename = filename
        self.smiles_column = SMILES_COLUMN
        self.y_column = TARGET_COLUMN

        self.data = None
        self.smiles = None
        self.targets = None
        self.indices = None

        self.target_map = {True: 1, False: 0}

    def __getitem__(self, item):
        return self.smiles[item], self.indices[item], self.targets[item]

    def get_processed_smiles_and_targets(self):
        self.data = self.load_train_data()
        list_of_smiles = self.data[self.smiles_column]
        processed_list_of_smiles = self.process_list_of_smiles(list_of_smiles)
        self.smiles = processed_list_of_smiles
        if self.y_column in list(self.data):
            targets = self.change_str_target_to_int(self.data[self.y_column])
            self.targets = targets
        self.indices = list(range(len(self.smiles)))
        return self.smiles, self.targets

    def load_train_data(self):
        path = os.path.join(self.data_dir, self.filename)
        data = pd.read_csv(path)
        return data

    def process_list_of_smiles(self, list_of_smiles: List[str]):
        processed_list_of_smiles = list(map(lambda x: self.remove_salts_and_canonicalized(x), list_of_smiles))
        return processed_list_of_smiles

    def remove_salts_and_canonicalized(self, smiles: str):
        remover = SaltRemover(defnData="[Cl,Br]")
        mol = Chem.MolFromSmiles(smiles)
        res = remover.StripMol(mol)
        processed_smiles = Chem.MolToSmiles(mol)
        return processed_smiles

    def change_str_target_to_int(self, targets: pd.Series):
        processed_targets = targets.map(self.target_map)
        return processed_targets.values

In [ ]:
import pandas as pd

def load_train_test_data():
    train_csv_path = os.path.join(DATA_PATH, TRAIN_FILE)
    test_csv_path = os.path.join(DATA_PATH, TEST_FILE)
    train_data = pd.read_csv(train_csv_path, index_col = 0)
    test_data = pd.read_csv(test_csv_path,index_col = 0)
    return train_data.rename(columns = {"Smiles":SMILES_COLUMN}), test_data.rename(columns = {"Smiles":SMILES_COLUMN})

## 1.1 Анализ данных, формулировка задачи машинного обучения

Молекулы представлены в виде [SMILES нотации](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system).

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/SMILES.png/450px-SMILES.png)

In [ ]:
train_data, test_data = load_train_test_data()
train_data.head()

,smiles,Active
0,COc1ccc2[nH]cc(CCN)c2c1,False
1,CCCN1CCC[C@H](c2cccc(O)c2)C1.Cl,False
2,O=C(NO)c1cnc(N2CCN(S(=O)(=O)c3ccc4ccccc4c3)CC2...,False
3,Nc1cccc(CNC(=O)c2ccc(Oc3ccc(OCc4cccc(F)c4)cc3)...,False
4,Fc1ccccc1CNCc1ccc(-c2ccnc3[nH]ccc23)cc1,False


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 92.3+ KB


In [ ]:
train_data[TARGET_COLUMN].value_counts()

False    5351
True      206
Name: Active, dtype: int64

## 1.2 Предобработка данных

In [ ]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover

In [ ]:
def remove_salts_and_canonicalized(smiles: str):
  remover = SaltRemover(defnData="[Cl,Br]")
  mol = Chem.MolFromSmiles(smiles)
  res = remover.StripMol(mol)
  processed_smiles = Chem.MolToSmiles(mol)
  return processed_smiles

In [ ]:
train_data[SMILES_COLUMN] = list(map(remove_salts_and_canonicalized, train_data[SMILES_COLUMN]))
test_data[SMILES_COLUMN] = list(map(remove_salts_and_canonicalized, test_data[SMILES_COLUMN]))

In [ ]:
def change_str_target_to_int(targets: pd.Series):
  target_map = {True: 1, False: 0}
  processed_targets = targets.map(target_map)
  return processed_targets.values

In [ ]:
train_data[TARGET_COLUMN] = change_str_target_to_int(train_data[TARGET_COLUMN])
test_data[TARGET_COLUMN] = change_str_target_to_int(test_data[TARGET_COLUMN])

In [ ]:
train_data.head()

,smiles,Active
0,COc1ccc2[nH]cc(CCN)c2c1,0
1,CCCN1CCC[C@H](c2cccc(O)c2)C1.Cl,0
2,O=C(NO)c1cnc(N2CCN(S(=O)(=O)c3ccc4ccccc4c3)CC2...,0
3,Nc1cccc(CNC(=O)c2ccc(Oc3ccc(OCc4cccc(F)c4)cc3)...,0
4,Fc1ccccc1CNCc1ccc(-c2ccnc3[nH]ccc23)cc1,0


## 1.3 Feature engineering

Молекулу можно представить в виде фингерпринта - вектора свойств, полученного по определенному алгоритму.

Мы будем считать фингерпринты при помощи библиотеки RDKit. Про различные фингерпринты и их описание можно почитать тут - https://www.rdkit.org/docs/GettingStartedInPython.html#fingerprinting-and-molecular-similarity

![](https://sun9-64.userapi.com/impf/_8Zy5WO6Mt0SIPx1YS02DeErAoZ0RHcwgc-kZg/Md98bNVzBg0.jpg?size=831x415&quality=96&sign=cb20481128a04ff523fd662dd0e604ab&type=album)


### Моргановские фингерпринты (ECFP)

![](https://d3i71xaburhd42.cloudfront.net/52adf3589e8b7b9855353e5815669258ef6e3405/6-Figure2-1.png)

In [ ]:
from enum import Enum
from functools import partial
from rdkit import Chem, DataStructs
from rdkit.DataStructs import ExplicitBitVect
from rdkit.Chem import AllChem, MACCSkeys
from typing import List


In [ ]:
class FingerprintsNames(Enum):
  ECFP4 = "morgan_2_2048"
  RDKitFP = "RDKFingerprint"
  TOPOTORSION = "topological_torsion"
  MACCS = "MACCSkeys"
  PATTERN = "PatternFingerprint"
  ATOMPAIR = "AtomPairFingerprint"



FINGERPRINTS_METHODS = {
    FingerprintsNames.ECFP4: partial(AllChem.GetMorganFingerprintAsBitVect, radius=2, nBits=2048),
    FingerprintsNames.RDKitFP: Chem.RDKFingerprint,#TODO
    FingerprintsNames.TOPOTORSION: AllChem.GetHashedTopologicalTorsionFingerprintAsBitVect,#TODO
    FingerprintsNames.MACCS: MACCSkeys.GenMACCSKeys,#TODO
    FingerprintsNames.PATTERN: Chem.PatternFingerprint,#TODO
    FingerprintsNames.ATOMPAIR: AllChem.GetHashedAtomPairFingerprintAsBitVect}#TODO


In [ ]:
fingerprint_type_name = FingerprintsNames.ECFP4
fingerprint_type_method = FINGERPRINTS_METHODS[fingerprint_type_name]

In [ ]:
def bit_vectors_to_numpy_arrays(fps: List[ExplicitBitVect]) -> np.array:
    output_arrays = [np.zeros((1,)) for i in range(len(fps))]
    _ = list(
        map(lambda fp_output_array: DataStructs.ConvertToNumpyArray(fp_output_array[0], fp_output_array[1]),
            zip(fps, output_arrays)))
    return np.asarray(output_arrays)

def get_np_array_of_fps(fp_type, smiles: List[str]):
    # Calculate the morgan fingerprint
    mols = [Chem.MolFromSmiles(m) for m in smiles]
    fp = list(map(fp_type, mols))
    return bit_vectors_to_numpy_arrays(fp)

In [ ]:
train_fp = get_np_array_of_fps(fp_type=fingerprint_type_method, smiles=train_data[SMILES_COLUMN])
test_fp = get_np_array_of_fps(fp_type=fingerprint_type_method, smiles=test_data[SMILES_COLUMN])

In [ ]:
y_train = train_data[TARGET_COLUMN]
y_test = test_data[TARGET_COLUMN]

# 2. Подготовка к обучению модели

## 2.1 Кросс-валидация

![](https://pubs.rsc.org/image/article/2018/SC/c7sc02664a/c7sc02664a-f3_hi-res.gif)

In [ ]:
from dgllife.utils import ScaffoldSplitter

In [ ]:
class ScaffoldCVSklearn:
    def __init__(self, data, k_folds):
        self.scaffold_splits = ScaffoldSplitter.k_fold_split(data, k=k_folds)

    def split(self):
        indices_splits = []
        for train_data, val_data in self.scaffold_splits:
          train_indices = train_data.indices
          val_indices = val_data.indices
          indices_splits.append((train_indices, val_indices))
        return indices_splits

    def convert_data_to_indices(self, dataset):
        indices = [index for index, row in dataset.iterrows()]
        return indices


In [ ]:
cv = ScaffoldCVSklearn(train_data, k_folds=3).split()

Start initializing RDKit molecule instances...
Creating RDKit molecule instance 1000/5557
Creating RDKit molecule instance 2000/5557
Creating RDKit molecule instance 3000/5557
Creating RDKit molecule instance 4000/5557
Creating RDKit molecule instance 5000/5557
Start computing Bemis-Murcko scaffolds.
Computing Bemis-Murcko for compound 1000/5557
Computing Bemis-Murcko for compound 2000/5557
Computing Bemis-Murcko for compound 3000/5557
Computing Bemis-Murcko for compound 4000/5557
Computing Bemis-Murcko for compound 5000/5557
Processing fold 1/3
Processing fold 2/3
Processing fold 3/3


## 2.2 Установка модели

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, nthread=1, use_label_encoder=False)

In [ ]:
params = {
        'max_depth': [10],
        'n_estimators': [100]
    }

## 2.3 Поиск параметров

In [ ]:
grid_search = GridSearchCV(xgb, param_grid=params, scoring='accuracy', n_jobs=4,
                               cv=cv, verbose=10000)

# Подбор параметров модели

In [ ]:
print('\n Start Grid Search')
grid_search.fit(train_fp, y_train)

In [ ]:

print('\n All results:')
print(grid_search.cv_results_)
print('\n Best estimator:')
print(grid_search.best_estimator_)
print('\n Best normalized score')
print(grid_search.best_score_)
print('\n Best hyperparameters:')
print(grid_search.best_params_)


 All results:
{'mean_fit_time': array([103.76662024]), 'std_fit_time': array([0.47663534]), 'mean_score_time': array([0.07499544]), 'std_score_time': array([0.03126068]), 'param_max_depth': masked_array(data=[10],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[100],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 10, 'n_estimators': 100}], 'split0_test_score': array([0.95412844]), 'split1_test_score': array([0.96706263]), 'split2_test_score': array([0.96976242]), 'mean_test_score': array([0.96365116]), 'std_test_score': array([0.00682319]), 'rank_test_score': array([1], dtype=int32)}

 Best estimator:
XGBClassifier(learning_rate=0.02, max_depth=10, nthread=1,
              use_label_encoder=False)

 Best normalized score
0.9636511647875509

 Best hyperparameters:
{'max_depth': 10, 'n_estimators': 100}


# Обучение и оценка модели

In [ ]:
xgb = XGBClassifier(max_depth=10, n_estimators=100, learning_rate=0.02,  nthread=1, use_label_encoder=False)

In [ ]:
xgb.fit(train_fp, y_train)

XGBClassifier(learning_rate=0.02, max_depth=10, nthread=1,
              use_label_encoder=False)

In [ ]:
test_predictions = xgb.predict(test_fp)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
score = f1_score(y_test, test_predictions)
print(f"Best model test f1 score is {round(score, 3)}")

Best model test f1 score is 0.265


# Задание (10 баллов + 3 бонусных)
1. (3 балла) Добавить решение проблемы несбалансированной классификации

Варианты:
* UnderSampling
* OverSampling
* SMOTE
* Внутренние инструменты модели (`scale_pos_weight`)

2. (2 балла) Использовать еще 2 вида фингерпринтов из `FingerprintsNames`

3. (3 балла) Получить f1-score на тестовом датасете больше 0.35

Варианты:
* Увеличить количество параметров в подборе гиперпараметров
* Использовать другие алгоритмы подбора гиперпараметров (например, [RandomizedSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html))
* Использовать другие модели (Random Forest, SVC, MLPClassifier, etc)

4. (2 балла) Логирование

В качестве финального результата предоставьте таблицу (можно `pd.DataFrame`) c колонками: Model, Fingerprint, Best Parameters, Mean Cross-Validation Score, Std Cross-Validation Score, Test Score 

Проанализируйте результаты: 
* Какие фингерпринты дали лучший результат?
* Какая модель дала лучший результат.
* Коррелируют ли скоры на кросс-валидации и тестовой выборке?

5. (Бонус +3 балла) Получить f1-score на тестовом датасете больше 0.45